## Extracting Audios From Videos

In [ ]:
import wave, math, contextlib
import speech_recognition as sr
from moviepy.editor import AudioFileClip

from os import listdir
from os.path import isfile, join
video_file_dir = './Eye Tracker Videos Raw'
audio_file_dir = './Eye Tracker Audio Converted'

allVideoFiles = [f for f in listdir(video_file_dir) if isfile(join(video_file_dir, f))]

for video_file_name in allVideoFiles:
    complete_video_file_dir = video_file_dir+'/'+video_file_name
    audio_dir = './Eye Tracker Audios Converted'
    part = video_file_name.split(' ')[1]
    print('- Converting Video To Audio for',part)
    transcribed_audio_file_name = audio_file_dir+'/'+"transcribed_speech_"+part+'.wav'
    audioclip = AudioFileClip(complete_video_file_dir)
    audioclip.write_audiofile(transcribed_audio_file_name, codec='pcm_s16le')
    from pydub import AudioSegment
    sound = AudioSegment.from_wav(transcribed_audio_file_name)
    sound = sound.set_channels(1)
    sound.export(transcribed_audio_file_name, format="wav")

print('- - Done Audio Seperation From Videos')

- Converting Video To Audio for P1_1
MoviePy - Writing audio in ./Eye Tracker Audio Converted/transcribed_speech_P1_1.wav


MoviePy - Done.
- Converting Video To Audio for P1_2from
MoviePy - Writing audio in ./Eye Tracker Audio Converted/transcribed_speech_P1_2from.wav


MoviePy - Done.
- Converting Video To Audio for P2
MoviePy - Writing audio in ./Eye Tracker Audio Converted/transcribed_speech_P2.wav


MoviePy - Done.
- Converting Video To Audio for P3
MoviePy - Writing audio in ./Eye Tracker Audio Converted/transcribed_speech_P3.wav


MoviePy - Done.
- Converting Video To Audio for P4_1
MoviePy - Writing audio in ./Eye Tracker Audio Converted/transcribed_speech_P4_1.wav


...

MoviePy - Done.
- Converting Video To Audio for P16
MoviePy - Writing audio in ./Eye Tracker Audio Converted/transcribed_speech_P16.wav


MoviePy - Done.
- Converting Video To Audio for P17_1
MoviePy - Writing audio in ./Eye Tracker Audio Converted/transcribed_speech_P17_1.wav


MoviePy - Done.
- Converting Video To Audio for P17_2
MoviePy - Writing audio in ./Eye Tracker Audio Converted/transcribed_speech_P17_2.wav


MoviePy - Done.
- Converting Video To Audio for P18
MoviePy - Writing audio in ./Eye Tracker Audio Converted/transcribed_speech_P18.wav


MoviePy - Done.
- - Done Audio Seperation From Videos


## Converting Audios To Text Timestamps

In [ ]:
import pandas as pd
class custom_Word:
    ''' A class representing a word from the JSON format for vosk speech recognition API '''

    def __init__(self, dict):
        '''
        Parameters:
          dict (dict) dictionary from JSON, containing:
            conf (float): degree of confidence, from 0 to 1
            end (float): end time of the pronouncing the word, in seconds
            start (float): start time of the pronouncing the word, in seconds
            word (str): recognized word
        '''

        self.conf = dict["conf"]
        self.end = dict["end"]
        self.start = dict["start"]
        self.word = dict["word"]

    def to_string(self):
        ''' Returns a string describing this instance '''
        return "{:20} from {:.2f} sec to {:.2f} sec, confidence is {:.2f}%".format(
            self.word, self.start, self.end, self.conf*100)

import wave
import json

from vosk import Model, KaldiRecognizer, SetLogLevel
# import Word as custom_Word

allAudioFiles = [f for f in listdir(audio_file_dir) if isfile(join(audio_file_dir, f))]
with pd.ExcelWriter('output_audio_to_text_timestamp.xlsx') as writer:
    for audio_filename in allAudioFiles:
        audio_filename_ = audio_filename
        audio_filename = audio_file_dir+'/'+audio_filename

        model_path = "models/vosk-model-en-us-0.22"
        print('& starting',audio_filename)

        # model = Model(model_path)
        model = Model(lang="en-us")
        # wf = wave.open(audio_filename, "w")
        # wf.setsampwidth(2)
        # wf.setnchannels(1)
        wf = wave.open(audio_filename, 'rb')
        if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getcomptype() != "NONE":
            print("Audio file must be WAV format mono PCM.")
            print('n Channels: ',wf.getnchannels())
            print('Sample Width: ', wf.getsampwidth())
            print('Comp Type: ',wf.getcomptype())
            # sys.exit(1)
        else:
            print('-- File Checked')
        rec = KaldiRecognizer(model, wf.getframerate())
        rec.SetWords(True)

        # get the list of JSON dictionaries
        results = []
        # recognize speech using vosk model
        while True:
            data = wf.readframes(4000)
            if len(data) == 0:
                break
            if rec.AcceptWaveform(data):
                part_result = json.loads(rec.Result())
                results.append(part_result)
        part_result = json.loads(rec.FinalResult())
        results.append(part_result)

        # convert list of JSON dictionaries to list of 'Word' objects
        print('Converting ...')
        list_of_Words = []
        for sentence in results:
            if len(sentence) == 1:
                # sometimes there are bugs in recognition
                # and it returns an empty dictionary
                # {'text': ''}
                continue
            for obj in sentence['result']:
                w = custom_Word(obj)  # create custom Word object
                list_of_Words.append(w)  # and add it to list

        wf.close()  # close audiofile

        # output to the screen
        conf = []
        end = []
        start = []
        wd = []
        for word in list_of_Words:
            # print(word.to_string())
            conf.append(word.conf)
            end.append(word.end)
            start.append(word.start)
            wd.append(word.word)
        pd.DataFrame({'confidence':conf, 'end':end, 'start':start, 'word':wd}).to_excel(writer, sheet_name=audio_filename_)

& starting ./Eye Tracker Audio Converted/transcribed_speech_P10.wav
-- File Checked
Converting ...
& starting ./Eye Tracker Audio Converted/transcribed_speech_P11.wav
-- File Checked
Converting ...
& starting ./Eye Tracker Audio Converted/transcribed_speech_P12.wav
-- File Checked
Converting ...
& starting ./Eye Tracker Audio Converted/transcribed_speech_P13.wav
-- File Checked
Converting ...
& starting ./Eye Tracker Audio Converted/transcribed_speech_P14.wav
-- File Checked
Converting ...
& starting ./Eye Tracker Audio Converted/transcribed_speech_P15.wav
-- File Checked
Converting ...
& starting ./Eye Tracker Audio Converted/transcribed_speech_P16.wav
-- File Checked
Converting ...
& starting ./Eye Tracker Audio Converted/transcribed_speech_P17_1.wav
-- File Checked


Exception ignored in: <function Wave_write.__del__ at 0x000001833F3F5430>
Traceback (most recent call last):
  File "c:\Users\hwx1162214\Anaconda3\lib\wave.py", line 326, in __del__
    self.close()
  File "c:\Users\hwx1162214\Anaconda3\lib\wave.py", line 444, in close
    self._ensure_header_written(0)
  File "c:\Users\hwx1162214\Anaconda3\lib\wave.py", line 464, in _ensure_header_written
    raise Error('sample width not specified')
wave.Error: sample width not specified
Exception ignored in: <function Wave_write.__del__ at 0x000001833F3F5430>
Traceback (most recent call last):
  File "c:\Users\hwx1162214\Anaconda3\lib\wave.py", line 326, in __del__
    self.close()
  File "c:\Users\hwx1162214\Anaconda3\lib\wave.py", line 444, in close
    self._ensure_header_written(0)
  File "c:\Users\hwx1162214\Anaconda3\lib\wave.py", line 466, in _ensure_header_written
    raise Error('sampling rate not specified')
wave.Error: sampling rate not specified


Converting ...
& starting ./Eye Tracker Audio Converted/transcribed_speech_P17_2.wav
-- File Checked
Converting ...
& starting ./Eye Tracker Audio Converted/transcribed_speech_P18.wav
-- File Checked
Converting ...
& starting ./Eye Tracker Audio Converted/transcribed_speech_P1_1.wav
-- File Checked
Converting ...
& starting ./Eye Tracker Audio Converted/transcribed_speech_P1_2from.wav
-- File Checked
Converting ...
& starting ./Eye Tracker Audio Converted/transcribed_speech_P2.wav
-- File Checked
Converting ...
& starting ./Eye Tracker Audio Converted/transcribed_speech_P3.wav
-- File Checked
Converting ...
& starting ./Eye Tracker Audio Converted/transcribed_speech_P4_1.wav
-- File Checked
Converting ...
& starting ./Eye Tracker Audio Converted/transcribed_speech_P4_2.wav
-- File Checked
Converting ...
& starting ./Eye Tracker Audio Converted/transcribed_speech_P5_1.wav
-- File Checked
Converting ...
& starting ./Eye Tracker Audio Converted/transcribed_speech_P5_2.wav
-- File Checked
